In [1]:
import pandas as pd
from glob import glob
from os import listdir, chdir
from re import compile as rcompile
from path import Path
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def read_transcript(file_name):
    file_names = glob(f'{file_name}*')
    ret = ''
    for fname in file_names:
        with open(fname) as fd:
            ret += fd.read()+'\n\n'
    return ret

reference_transcripts_dir = '../data/transcripts'
google_transcripts_dir = '../data/google_transcripts'

reference_transcripts_files = listdir(reference_transcripts_dir)
google_transcripts_files = listdir(google_transcripts_dir)
reference_transcripts_files.sort()
google_transcripts_files.sort()

In [3]:
print(reference_transcripts_files)
print(google_transcripts_files)

['.ipynb_checkpoints', '3101.txt', '3102.txt', '3103.txt', '3104.txt', '3105.txt', '3106.txt', '3107.txt', '3108.txt', '3109.txt', '3110.txt', '3111.txt', '3112.txt', '3113.txt', '3114.txt', '3115.txt', '3116.txt', '3117.txt', '3118.txt', '3119.txt', '3120.txt', '3121.txt', '3122.txt', '3123.txt', '3124.txt', '3125.txt', '3126.txt', '3127.txt', '3128.txt', '3129.txt', '3130.txt', '3131.txt', '3132.txt', '3201.txt', '3202.txt', '3203.txt', '3204.txt', '3205.txt', '3206.txt', '3207.txt', '3208.txt', '3209.txt', '3210.txt', '3211.txt', '3212.txt', '3213 (missing page 2).txt', '3214.txt', '3215 (missing page 2).txt', '3216.txt', '3217.txt', '3218.txt', '3219.txt', '3220.txt', '3221.txt', '3222.txt', '3223.txt', '3224.txt', '3225.txt', '3226.txt', '3227.txt', '3228.txt', '3229.txt', '3230.txt', '3231.txt', '3232.txt', '3234.txt', '3235.txt', '3236.txt', '3237.txt', '3238.txt', '3239.txt', '3240.txt', '3241.txt', '3243.txt', '3244.txt', '3245.txt', '3246.txt', '3247.txt', '3248.txt', '5101.t

In [4]:
#Some of the reference transcripts have missing information and those files have
#filename with '(missing page <number>)' substrings - filter these files out for
#now.
rex = rcompile('[0-9]+.txt')
reference_transcripts_file_roots = [x.split('.')[0] for x in reference_transcripts_files if rex.match(x) is not None]
print(reference_transcripts_file_roots)

['3101', '3102', '3103', '3104', '3105', '3106', '3107', '3108', '3109', '3110', '3111', '3112', '3113', '3114', '3115', '3116', '3117', '3118', '3119', '3120', '3121', '3122', '3123', '3124', '3125', '3126', '3127', '3128', '3129', '3130', '3131', '3132', '3201', '3202', '3203', '3204', '3205', '3206', '3207', '3208', '3209', '3210', '3211', '3212', '3214', '3216', '3217', '3218', '3219', '3220', '3221', '3222', '3223', '3224', '3225', '3226', '3227', '3228', '3229', '3230', '3231', '3232', '3234', '3235', '3236', '3237', '3238', '3239', '3240', '3241', '3243', '3244', '3245', '3246', '3247', '3248', '5101', '5102', '5103', '5104', '5105', '5106', '5107', '5108', '5109', '5110', '5111', '5112', '5113', '5114', '5115', '5116', '5117', '5118', '5119', '5120', '5121', '5122', '5123', '5124', '5125', '5126', '5129', '5130', '5131', '5132', '5202', '5203', '5204', '5205', '5206', '5207', '5208', '5209', '5210', '5213', '5214', '5215', '5216', '5217', '5218', '5219', '5220', '5221', '5222',

In [5]:

with Path(reference_transcripts_dir):
    reference_transcripts = [read_transcript(f'{x}.txt') for x in reference_transcripts_file_roots]

print(len(reference_transcripts),'\n', reference_transcripts[0],'\n', reference_transcripts[1])

with Path(google_transcripts_dir):
    google_transcripts = [read_transcript(x) for x in reference_transcripts_file_roots]
    
print(len(google_transcripts),'\n', google_transcripts[0],'\n', google_transcripts[1])


165 
 Once apon a time there was a girl named Mary. On a warm 
sunny day Mary was walking through the woods near hear nouse 
to look for some critters to take pictures of. She loved animals 
and nature all her life even though she was only nine years old 
She thinks that she is going to die soon. She does go to school 
but she isn't that smart. For example Mary recycled a pie 
even though the pie was not even bitten. Another thing she did 
was leave her muffin in the woods while she was traveling with  
her family to Andrea's house which is one of Mary's friends at 
school and Mary's muffin was eaten by a bear that sniffed it :q!
:
from far away. Yes. Mary could be a little weird but that is 
just how she is. Back to the real story now. Mary found a 
fox, took a picture she also found a rabbit took a picture 
she found a squirrel and took a picture before it ran away. 
But sooner or later she saw two eyes peeking through a bush 
on the side of the path she was walking on. Then the 
cre

In [6]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english',
                        ngram_range=(1,2),
                        min_df=1
                       )
# Create a vocabulary and get word counts per document
ref_dtm = tfidf.fit_transform(reference_transcripts)

features = tfidf.get_feature_names()
#display(len(features), features[:50])

# Get feature names to use as dataframe column headers
ref_dtm = pd.DataFrame(ref_dtm.todense(), columns=features)
print(ref_dtm.shape)
display(ref_dtm.head())


(165, 21876)


,00,00 meet,00 morning,00 said,00 star,00 telling,00 time,000,000 000,000 5203,...,zoe told,zoe wasn,zoe worms,zoomed,zoomed planted,zoomed shop,zrm,zrm jedi,zzzzzz,zzzzzz wake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051429,0.051429
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [7]:
ishape = ref_dtm.shape[1]
# Create Model 
input_img = Input(shape=(ishape, ))

x = Dense(512)(input_img)

x = Dense(256)(x)

x = Dense(128)(x)

encoded = Dense(64)(x)

x = Dense(128)(encoded)

x = Dense(256)(x)

x = Dense(512, activation='sigmoid')(x)
decoded = Dense(ishape, activation='sigmoid')(x)


rmodel = Model(input_img, decoded)
rmodel.compile(loss='mse', optimizer=Adam(learning_rate=0.01))

rmodel.fit(ref_dtm, ref_dtm, batch_size=512, epochs=5)

Epoch 1/5
1/1 [==============================] - 1s 556ms/step - loss: 0.2506
Epoch 2/5
1/1 [==============================] - 0s 133ms/step - loss: 0.0263
Epoch 3/5
1/1 [==============================] - 0s 139ms/step - loss: 0.0022
Epoch 4/5
1/1 [==============================] - 0s 133ms/step - loss: 9.5287e-05
Epoch 5/5
1/1 [==============================] - 0s 138ms/step - loss: 4.5479e-05


In [8]:
encoder = Model(input_img, encoded)

encoded_ref_dtm = encoder.predict(ref_dtm)
print(encoded_ref_dtm.shape)
display(encoded_ref_dtm)

(165, 64)


array([[-1.003536  ,  1.0493752 ,  1.6007402 , ..., -0.38971758,
         1.1976781 ,  0.885774  ],
       [-0.9935966 ,  1.0859337 ,  1.5872387 , ..., -0.3522619 ,
         1.1731225 ,  0.86557317],
       [-0.96297735,  1.0114995 ,  1.5602409 , ..., -0.36948264,
         1.1545056 ,  0.85654515],
       ...,
       [-0.9603699 ,  1.000963  ,  1.4966521 , ..., -0.33580783,
         1.1373684 ,  0.8458786 ],
       [-0.92219543,  0.9894248 ,  1.4704257 , ..., -0.33309492,
         1.0910069 ,  0.8046113 ],
       [-0.90533465,  0.9771337 ,  1.4496839 , ..., -0.320725  ,
         1.0763164 ,  0.7914262 ]], dtype=float32)

In [9]:
def get_vector(doc):
    vec = tfidf.transform([doc]).todense()
    return encoder.predict(vec)

In [10]:
print(get_vector(google_transcripts[0]))

[[-1.0019499   1.054158    1.5928526   0.35311574  0.21105306 -0.61814034
  -1.2876871   0.07740943 -0.632058    1.9220977   0.4431421  -0.44661903
  -0.28988427 -0.80466485  0.65710664  0.04627845 -0.41365033  0.32011217
  -0.55627507 -1.205483   -2.1990309  -0.23523569 -0.2529007   0.10884329
  -0.19295302 -0.14037667  0.57748586 -0.6100995   0.55940413  0.10510726
  -0.2634164  -0.2594606   0.1469222  -0.74175507 -1.0035548  -0.4628877
   1.603921    0.7719165  -0.03361821 -1.146816   -0.6604583   0.4995437
  -1.2742559  -0.03848452  0.5715377  -0.82754976  0.46475062  1.6885381
   0.69039476  0.90078163 -0.04280966  0.22047238  0.6627176   0.41684788
  -0.4136641   0.3027792  -1.2714778   1.087907   -0.4236136  -0.16258168
  -0.38486195 -0.3883356   1.1981865   0.8876763 ]]


In [11]:
def get_encoded_dtm(transcripts):
    dtm = tfidf.transform(transcripts).todense()
    print(dtm.shape)
    encoded_dtm = encoder.predict(dtm)
    print(encoded_dtm.shape)
    display(encoded_dtm)
    return encoded_dtm

encoded_google_dtm = get_encoded_dtm(google_transcripts)
print(encoded_google_dtm.shape)
display(encoded_google_dtm)

(165, 21876)
(165, 64)


array([[-1.0019501 ,  1.0541575 ,  1.592853  , ..., -0.38833568,
         1.1981866 ,  0.88767606],
       [-0.99313176,  1.0679523 ,  1.5787816 , ..., -0.34595945,
         1.1624902 ,  0.8469339 ],
       [-0.9680624 ,  1.0142884 ,  1.5717391 , ..., -0.3684737 ,
         1.1675831 ,  0.8540629 ],
       ...,
       [-0.9417632 ,  0.98466694,  1.4723846 , ..., -0.3268375 ,
         1.113625  ,  0.8360641 ],
       [-0.929472  ,  0.9976619 ,  1.49686   , ..., -0.34000194,
         1.1067028 ,  0.81129956],
       [-0.85890645,  0.89839345,  1.3502383 , ..., -0.29596093,
         1.0245094 ,  0.7377031 ]], dtype=float32)

(165, 64)


array([[-1.0019501 ,  1.0541575 ,  1.592853  , ..., -0.38833568,
         1.1981866 ,  0.88767606],
       [-0.99313176,  1.0679523 ,  1.5787816 , ..., -0.34595945,
         1.1624902 ,  0.8469339 ],
       [-0.9680624 ,  1.0142884 ,  1.5717391 , ..., -0.3684737 ,
         1.1675831 ,  0.8540629 ],
       ...,
       [-0.9417632 ,  0.98466694,  1.4723846 , ..., -0.3268375 ,
         1.113625  ,  0.8360641 ],
       [-0.929472  ,  0.9976619 ,  1.49686   , ..., -0.34000194,
         1.1067028 ,  0.81129956],
       [-0.85890645,  0.89839345,  1.3502383 , ..., -0.29596093,
         1.0245094 ,  0.7377031 ]], dtype=float32)

In [12]:
cosine_similarity([encoded_ref_dtm[0]], [encoded_google_dtm[0]])

array([[0.99998903]], dtype=float32)

In [22]:
def get_cossim(encoded_dtm):
    cossim = []
    for idx,ref in enumerate(encoded_ref_dtm):
        cossim.append(cosine_similarity([ref], [encoded_dtm[idx]])[0][0]) 
    return cossim

cossim = get_cossim(encoded_google_dtm)
print(cossim)
mean_cossim = sum(cossim)/len(cossim)
print(mean_cossim, round((1-mean_cossim)*1e5,4))

[0.99998903, 0.99995464, 0.9999215, 0.999747, 0.99994886, 0.99998665, 0.9999648, 0.999956, 0.9999603, 0.9999411, 0.99998903, 0.9999451, 0.99995214, 0.9999762, 0.99993694, 0.99995935, 0.99980515, 0.9999228, 0.9999206, 0.9999718, 0.9999655, 0.9999904, 0.99995375, 0.9998677, 0.99999833, 0.99997795, 0.9999949, 0.99991816, 0.9999572, 0.99997425, 0.9999852, 0.999977, 0.9999342, 0.99995804, 0.9999897, 0.9999698, 0.9999871, 0.9999636, 0.9999765, 0.99997467, 0.9999826, 0.99996483, 0.9999898, 0.9999624, 0.99999356, 0.99992436, 0.99990404, 0.9999635, 0.9999894, 0.99997014, 0.99995226, 0.9999166, 0.999961, 0.9999919, 0.999974, 0.9999265, 0.99997854, 0.99980104, 0.9999692, 0.99998367, 0.9999747, 0.9999778, 0.99996483, 0.99967754, 0.99974066, 0.9999822, 0.99996233, 0.9999822, 0.99979866, 0.9999949, 0.9999254, 0.9999559, 0.99996847, 0.9999592, 0.9999809, 0.99999726, 0.99993193, 0.99993765, 0.9999746, 0.999948, 0.99993324, 0.99999017, 0.9999794, 0.99996257, 0.99988425, 0.99987054, 0.9999888, 0.9999835

In [14]:
from joblib import dump
MODELS_DIR = '../models/'

TFIDF = MODELS_DIR+'tfidf.pkl'
ENCODED_REF_DTM = MODELS_DIR+'encoded_ref_dtm.pkl'
ENCODER = MODELS_DIR+'encoder.h5'

dump(tfidf, TFIDF)
encoder.save(ENCODER)
dump(encoded_ref_dtm, ENCODED_REF_DTM)

['../models/encoded_ref_dtm.pkl']